In [1]:
import pandas as pd
import numpy as np
import functools as ft
import itertools as it
import multiprocessing as mp
import matplotlib.pyplot as plt
from collections import Counter
%matplotlib inline

In [8]:
df = pd.read_csv('../data/go_synonym_data.txt', header=None, sep='\t', low_memory=False,)
df.columns = ['ID', 'Gene/Product', 'Name', 'GO Class Labels', 'Synonyms']
print(df.shape)
df.head(3)

(19712, 5)


,ID,Gene/Product,Name,GO Class Labels,Synonyms
0,UniProtKB:P25788,PSMA3,Proteasome subunit alpha type-3,protein deubiquitination|endopeptidase activit...,HC8|PSC8
1,UniProtKB:P25787,PSMA2,Proteasome subunit alpha type-2,protein deubiquitination|endopeptidase activit...,HC3|PSC3
2,UniProtKB:P25786,PSMA1,Proteasome subunit alpha type-1,protein deubiquitination|endopeptidase activit...,HC2|NU|PROS30|PSC2


Below is an example of what the labels for the BOW are

In [5]:
print(df.iloc[0,3])

protein deubiquitination|endopeptidase activity|proteasome complex|threonine-type endopeptidase activity|synapse|post-translational protein modification|nucleoplasm|ubiquitin-dependent protein catabolic process|nucleus|proteasome-mediated ubiquitin-dependent protein catabolic process|protein binding|ubiquitin protein ligase binding|extracellular exosome|cytoplasm|proteasome core complex|viral process|proteasome core complex, alpha-subunit complex|cytosol|regulation of endopeptidase activity|proteasomal ubiquitin-independent protein catabolic process|proteasomal protein catabolic process


In [9]:
print(len(df['ID'].unique()))

19712


In [7]:
print(len(df['Gene/Product'].unique()))

19659


In [16]:
df.set_index('Gene/Product')[df.set_index('Gene/Product').index.duplicated('last')]

,ID,Name,GO Class Labels,Synonyms
Gene/Product,,,,
PDE8B,UniProtKB:O95263,High affinity cAMP-specific and IBMX-insensiti...,G-protein coupled receptor signaling pathway|m...,PIG22
MUC21,UniProtKB:A0A140T8X8,Mucin-21,negative regulation of cell adhesion|negative ...,NaN
ARHGEF18,UniProtKB:A0A087WZG4,Rho guanine nucleotide exchange factor 18,nucleic acid binding|regulation of Rho protein...,NaN
GABARAP,UniProtKB:O95166,Gamma-aminobutyric acid receptor-associated pr...,smooth endoplasmic reticulum|Golgi membrane|ex...,FLC3B|HT004
KRTAP9-6,UniProtKB:A0A140TA67,Keratin-associated protein 9-6,keratin filament,NaN
VARS2,UniProtKB:A0A140TA40,"Valine--tRNA ligase, mitochondrial",valine-tRNA ligase activity|ATP binding|valyl-...,NaN
EIF3F,UniProtKB:A0A0D9SEZ9,Eukaryotic translation initiation factor 3 sub...,translation initiation factor activity|eukaryo...,UniProtKB:A0A0D9SEZ9|PTN002576045
SCO2,UniProtKB:A0A1W2PP80,"Protein SCO2 homolog, mitochondrial",phosphorylase activity|pyrimidine nucleobase m...,NaN
MUC21,UniProtKB:A0A0G2JKD1,Mucin-21,negative regulation of cell adhesion|negative ...,NaN


In [15]:
df.set_index('Gene/Product')[df.set_index('Gene/Product').index.duplicated()]

,ID,Name,GO Class Labels,Synonyms
Gene/Product,,,,
MUC21,UniProtKB:A0A0G2JKD1,Mucin-21,negative regulation of cell adhesion|negative ...,NaN
LRTOMT,UniProtKB:Q8WZ04,Transmembrane O-methyltransferase,catechol O-methyltransferase activity|dopamine...,COMT2|TOMT|PP7517
ZNF280D,UniProtKB:H0YL38,Zinc finger protein 280D,nucleic acid binding,NaN
ZC3H11B,UniProtKB:A0A1B0GTU1,Zinc finger CCCH domain-containing protein 11B,metal ion binding,ZC3HDC11B
TUBA3C,UniProtKB:P0DPH7,Tubulin alpha-3C chain,GTP binding|cytoskeleton organization|cytoplas...,TUBA2
VSIG8,UniProtKB:P0DPA2,V-set and immunoglobulin domain-containing pro...,RNA binding|integral component of membrane|int...,C1orf204
TJP2,UniProtKB:Q9UDY2,Tight junction protein ZO-2,establishment of endothelial intestinal barrie...,X104|ZO2
NSG1,UniProtKB:P42857,Neuronal vesicle trafficking-associated protein 1,clathrin light chain binding|lateral plasma me...,D4S234|NEEP21
SCO2,UniProtKB:O43819,"Protein SCO2 homolog, mitochondrial",protein disulfide oxidoreductase activity|cell...,NaN


In [34]:
kinase_network_df = pd.read_csv('../data/Full_Kinome_Network_Compiled_no_header.txt', header=None, sep='\t')
known_kinases = list(set(kinase_network_df[0]) | set(kinase_network_df[1]))
print(len(known_kinases))

473


In [35]:
df[df['Gene/Product']=='RAF1']

,ID,Gene/Product,Name,GO Class Labels,Synonyms
18381,UniProtKB:P04049,RAF1,RAF proto-oncogene serine/threonine-protein ki...,nuclear speck|Golgi apparatus|face development...,RAF


In [20]:
df.dtypes

ID                 object
Gene/Product       object
Name               object
GO Class Labels    object
Synonyms           object
dtype: object

In [29]:
print(len(list(filter(lambda x: x != 'nan', it.chain.from_iterable(df['Synonyms'].apply(lambda x: str(x).split('|')))))))
print(len(set(filter(lambda x: x != 'nan', it.chain.from_iterable(df['Synonyms'].apply(lambda x: str(x).split('|')))))))

23978
23297


In [36]:
set_of_synonyms_present = set(filter(lambda x: x != 'nan', it.chain.from_iterable(df['Synonyms'].apply(lambda x: str(x).split('|')))))
set_of_gene_products = set(df['Gene/Product'].values)

In [39]:
missing = []

for k in known_kinases:
    if k not in set_of_synonyms_present and k not in set_of_gene_products:
        missing.append(k)

In [40]:
len(missing)

155

In [42]:
'ACV1C' in set_of_gene_products or 'ACV1C' in set_of_synonyms_present

False

In [41]:
'ACVR1C' in set_of_gene_products # the ACV1C protein is not a synonym

True

In [44]:
alias = pd.read_excel('../data/KINASESmasterlist_w_Aliases.xlsx', header = 0)

In [45]:
alias.head(5)

,Uniprot Protein,MS Gene,RNAseq Gene,RNAseq Accession,Family,Mouse Uniprot Protein,Mouse RNAseq gene,Mouse RNAseq Accession,Kinome Render Tree Name,Aliases (Conservative),...,Aliases,description,other_designations,Entrez_Symbol,Old_Name,Entrez_Synonyms,Entrez_other_designations,Unnamed: 18,Gene Symbol,Gene Symbol and Synonyms
0,AAK1,AAK1,AAK1,22848.0,Other,Aak1,Aak1,269774,AAK1,"AAK1,",...,NaN,AP2 associated kinase 1,adaptor-associated kinase 1,AAK1,AAK1,KIAA1048|MGC138170,adaptor-associated kinase 1,NaN,AAK1,AAK1
1,AAPK1,PRKAA1,PRKAA1,5562.0,CAMK,Aapk1,Prkaa1,105787,AMPK[alpha]1,"PRKAA1, AMPKa1",...,"AMPK, AMPKa1","protein kinase, AMP-activated, alpha 1 catalyt...","5'-AMP-activated protein kinase, catalytic alp...",PRKAA1,AMPKa1,AMPK|AMPKa1|MGC33776|MGC57364,"5'-AMP-activated protein kinase, catalytic alp...",NaN,PRKAA1,"AMPK, PRKAA1, AMPKa1"
2,AAPK2,PRKAA2,PRKAA2,5563.0,CAMK,Aapk2,Prkaa2,108079,AMPK[alpha]2,"PRKAA2, AMPK2, AMPKa2, PRKAA",...,"AMPK, AMPK2, AMPKa2, PRKAA","protein kinase, AMP-activated, alpha 2 catalyt...","5'-AMP-activated protein kinase, catalytic alp...",PRKAA2,AMPKa2,AMPK|AMPK2|PRKAA,"5'-AMP-activated protein kinase, catalytic alp...",NaN,PRKAA2,"AMPK2, AMPK, PRKAA, AMPKa2, PRKAA2"
3,ABL1,ABL1,ABL1,25.0,TK,Abl1,Abl1,11350,Abl,"ABL1,ABL, JTK7, bcr/abl, c-ABL, c-ABL1, v-abl",...,"ABL, JTK7, bcr/abl, c-ABL, c-ABL1, p150, v-abl","ABL proto-oncogene 1, non-receptor tyrosine ki...",Abelson tyrosine-protein kinase 1|bcr/c-abl on...,ABL1,ABL,ABL|JTK7|c-ABL|p150|v-abl,Abelson murine leukemia viral (v-abl) oncogene...,NaN,ABL1,"c-ABL1, p150, ABL, JTK7, v-abl, bcr/abl, c-ABL..."
4,ABL2,ABL2,ABL2,27.0,TK,Abl2,Abl2,11352,Arg,"ABL2,ABLL, ARG",...,"ABLL, ARG","ABL proto-oncogene 2, non-receptor tyrosine ki...","abelson-related gene protein|c-abl oncogene 2,...",ABL2,ARG,ABLL|ARG,Abelson murine leukemia viral (v-abl) oncogene...,NaN,ABL2,"ABL2, ABLL, ARG"


In [49]:
for sym in alias['Symbol']:
    if sym not in set_of_gene_products and sym not in set_of_synonyms_present and sym is not np.nan:
        print(sym)